# ONNX 基础概念


ONNX 是一个开源格式，旨在实现机器学习模型在各种框架之间的互操作性。它作为一个桥梁，使得在一个框架中构建的模型（如 TensorFlow、PyTorch 等）能够在另一个框架中使用，从而确保更广泛的兼容性和更便捷的部署。

接下来，我们将对 ONNX 的基础概念进行介绍。ONNX 中的核心数据结构是 ModelProto，它组织了神经网络的结构和元数据。ONNX 的组成部分为：

1. **模型（Model）**：ONNX 模型是完整的机器学习模型定义，包含计算图、权重、元数据等。它由一个或多个图（Graph）组成，通常只有一个主图。
2. **图（Graph）**：图表示模型的计算流程，定义从输入到输出的整个计算路径。图包含所有的计算节点（Nodes）和它们之间的数据流。
3. **节点（Nodes）**：节点是图中的基本计算单元，代表具体的操作（Operation）。节点通过操作符（Operator）执行计算，接收输入张量并生成输出张量（Tensors）。
4. **操作符（Operators）**：操作符定义了节点执行的具体计算操作，如加法、卷积等，决定了节点如何处理输入张量生成输出张量。
5. **张量（Tensors）**：张量是数据在 ONNX 模型中传递的形式，表示多维数组的数据结构，用于在节点之间传递数据。
6. **初始值（Initializers）**：初始值是预设的常量张量，通常用于表示模型的权重或固定参数，在推理时作为常量输入给节点。
7. **输入（Inputs）和输出（Outputs）**：输入代表模型接收的外部数据，输出是模型计算的结果。它们是图计算的起点和终点。
8. **元数据（Metadata）**：元数据提供模型的额外信息，如作者、版本等，用于描述模型背景，不参与计算。

# 加载模型

In [2]:
! wget https://labfile.oss.aliyuncs.com/courses/40981/model.onnx

--2025-03-22 13:41:54--  https://labfile.oss.aliyuncs.com/courses/40981/model.onnx
Resolving labfile.oss.aliyuncs.com (labfile.oss.aliyuncs.com)... 47.110.177.159
Connecting to labfile.oss.aliyuncs.com (labfile.oss.aliyuncs.com)|47.110.177.159|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 980596 (958K) [application/octet-stream]
Saving to: 'model.onnx'

     0K .......... .......... .......... .......... ..........  5% 16.2M 0s
    50K .......... .......... .......... .......... .......... 10% 2.28M 0s
   100K .......... .......... .......... .......... .......... 15% 2.08M 0s
   150K .......... .......... .......... .......... .......... 20% 13.3M 0s
   200K .......... .......... .......... .......... .......... 26% 3.07M 0s
   250K .......... .......... .......... .......... .......... 31% 6.94M 0s
   300K .......... .......... .......... .......... .......... 36% 12.7M 0s
   350K .......... .......... .......... .......... .......... 41% 14.0M 0s
   400K 

In [3]:
import onnx

model = onnx.load("model.onnx")
type(model)

onnx.onnx_ml_pb2.ModelProto

# 模型可视化

模型可视化对于理解复杂的神经网络结构非常有帮助，尤其是在 ONNX 格式下。通过模型可视化，开发者可以直观地观察神经网络的层次结构、每一层的输入输出张量形状、权重的分布情况，以及各层之间的连接关系。这不仅有助于调试模型，还能更好地理解模型的设计思路和计算流程。

对于 ONNX 模型的可视化，有多种方式，最简单的是直接用 print 进行打印

In [ ]:
print(model)

打印输出的内容是一个 ModelProto 对象的字符串表示形式。这个字符串表示会包含模型的很多细节信息，前面介绍到的 ONNX 的组成部分，在其中都会展示。

但这种方法打印的无关信息太多，同时还存在 raw_data: "=p\366<\211\341\327\2749\... 这种以原始字节流形式存储的数据。由于这些数据直接存储为二进制，因此在打印时显示为乱码或特殊字符。

通常，对模型可视化我们只关心模型的结构，那么只要可视化计算图的信息就可以了，我们先获取模型的计算图

In [ ]:
model_graph = model.graph
print(model_graph.input)
print(model_graph.output)

可以看到，模型的输入输出都是一个一维的列表，这意味着这个模型是个单输入，单输出的模型。

输入的维度（shape）为 batch_size, 3, 224, 224 ， batch_size 是一个变量，意味着这个模型可以接受任意数量的图片，而 3, 224, 224 是常见的图片分类输入尺寸，分别对应图片的通道数、高度和宽度。

输出维度为 batch_size, 100 ，意味着这是一个具有 100 个类别标签的模型。

我们再来看模型的中间部分，也就是可视化所有的节点

In [ ]:
for node in model_graph.node:
    print(f"Node name: {node.name}")  # 节点名称
    print(f"Node operation: {node.op_type}")  # 节点操作类型
    print(f"Node inputs: {node.input}")  # 节点输入
    print(f"Node outputs: {node.output}")  # 节点输出
    print("\n")

此外，除了以上的可视化方法，还有其他的第三方的库，可以更加方便地可视化查看模型结构。如 Netron，这是一款用于神经网络模型可视化的开源工具，可以直观地查看神经网络模型的结构、层次关系、参数信息和数据流，这对于理解和调试复杂的深度学习模型非常有帮助。

In [ ]:
# ! netron model.onnx